In [2]:
# Scispacy is a library to preprocess biomedical/scientific text and it includes different variants of models like en_core_sci_sm,
# en_core_sci_md,en_core_sci_lg for query retrival

##################### Best Match 25 iterations ########################

import scispacy
import spacy
import pandas as pd
import json
import numpy as np
from tqdm import tqdm

In [3]:
with open('research_papers_cleaned.json') as j:
    papers_covid=json.loads(j.read())


In [257]:
# convert json data to csv file
df = pd.DataFrame(papers_covid)
df = df.rename(columns={0:'Headings',1:'Abstract',2:'Summary'})
df

,Headings,Abstract,Summary
0,Detection of viral respiratory pathogens in mi...,Community-ARI and inpatient-ARI was also combi...,community-ari inpatient-ari combine mild sever...
1,Detection of dicistroviruses RNA in blood of f...,Fever is the leading cause of paediatric outpa...,fever leading cause paediatric outpatient cons...
2,Anti-Respiratory Syncytial Virus Activity of P...,The herbs Plantago asiatica and Clerodendrum t...,herb plantago asiatica clerodendrum trichotomu...
3,Do corticosteroids reduce the mortality of inf...,Introduction: Corticosteroids are used empiric...,introduction corticosteroid empirically influe...
4,Lessons from a one-year hospital-based surveil...,Background: Surveillance of severe acute respi...,background surveillance severe acute respirato...
...,...,...,...
19810,Decoupling deISGylating and deubiquitinating a...,Coronavirus papain-like proteases (PLPs or PLp...,coronavirus papain-like protease plp plpro enc...
19811,Organoids -New Models for Host-Helminth Intera...,Organoids are multicellular culture systems th...,organoid multicellular culture system replicat...
19812,Influenza diagnosis and vaccination in Poland ଝ,In Poland between several thousand and several...,poland thousand million case influenza suspect...
19813,Coronavirus-induced ER stress response and its...,Coronavirus replication is structurally and fu...,coronavirus replication structurally functiona...


In [79]:
# print title , short abstract(cleaned summary) and abstract
titles = [paper[0] for paper in papers_covid]
summary = [paper[2] for paper in papers_covid]
abstract = [paper[1] for paper in papers_covid]


In [128]:
# so now we have titles,abstract and summary of abstract
# now load lg model and define find ranking of queries in docs using bm25
import rank_bm25
# import bm25 index library
from rank_bm25 import BM25Okapi 
# here BM25Okapi is used for ranking of query in a given document

In [202]:
nlp = spacy.load('en_core_sci_lg')

In [258]:
Abstract = np.array(df['Abstract'])

In [259]:
Abstract

array(['Community-ARI and inpatient-ARI was also combined with mild and severe cases of influenza from a historical prospective study as mild-ARI and severe-ARI respectively to evaluate the performance of clinical case definitions. We analysed 130 community-ARI and 140 inpatient-ARI episodes (5 inpatient-ARI excluded because multiple pathogens were detected), involving 138 and 207 samples respectively. Detection by PCR declined with days post-onset for influenza virus; decrease was faster for community-ARI than for inpatient-ARI. No such patterns were observed for noninfluenza respiratory virus infections. PathChip added substantially to viruses detected for community-ARI only. Clinical case definitions discriminated influenza from other mild-ARI but performed poorly for severe-ARI and for older participants. Rational strategies for diagnosis and surveillance of influenza and other respiratory virus must acknowledge the differences between ARIs presenting in community and hospital sett

In [279]:
query = 'Here coronavirus is attacking respiratory organs. We need to understand what countries or locations does it has same symptoms'
tokenized_query = ['sars-cov', 'sars', 'coronavirus', 'ace2', 'coronaviruses', 'ncov', 'covid-19', 'wuhan', 'spike', 'sars-cov-2','outbreak','drug','prevention','china','symptoms','outbreak','deaths','pandemic']

In [280]:
# first check text ranking algorithm BM25
# Now what BM25 take as an input-> it takes tokenized corpus as an input and then search for a given tokenized query in documents.
# In our case our corpus is abstract in which we wanna find our query, it's just a text ranking search algorithm, nothing more
# than that.
tokenized_abstract = [str(doc).split(' ') for doc in Abstract] # it basically outputs tokens of each doc in abstract

bm25 = BM25Okapi(tokenized_abstract)
scores = bm25.get_scores(tokenized_query)


In [286]:
indexes =pd.DataFrame(data={'indexes':np.argsort(scores)[::-1][:20]})
score_index = pd.DataFrame(data={'scores':np.array([scores[index] for index in np.argsort(scores)[::-1][:20]])})
result_scores = pd.concat([indexes,score_index],axis=1)
# make a dataframe of columns named index and scores


In [287]:
result_scores

,indexes,scores
0,19343,24.255764
1,7944,21.227668
2,16229,21.184994
3,3869,20.596164
4,17553,20.504333
5,5341,18.363689
6,14777,18.263492
7,6467,18.241438
8,8945,17.738073
9,16174,17.615747


In [302]:
Abstract = [str(doc) for doc in Abstract]
type(Abstract)

list

In [289]:
# now get best of the documents which matches given query
best_doc = bm25.get_top_n(tokenized_query,Abstract,n=20)

In [290]:
best_doc

['Severe acute respiratory syndrome (SARS) first appeared in November 2002 and ultimately resulted in 8096 probable human infections and 774 deaths worldwide. By July 2003, the global outbreak was declared over. A new coronavirus, SARSassociated coronavirus (SARS-CoV), was identified as the causative agent; this virus appeared to have a zoonotic origin, as genetically similar coronaviruses have been identified in several animal species. The global response to the outbreak was extensive. Within a short period, the pathogen had been identified, new diagnostic tests were developed, surveillance systems were created, infection control and prevention measures were instituted, and transmission among humans stopped. It is unclear if and when person-to-person SARS-CoV transmission will reappear. However, procedures have been established by public health organizations, including the World Health Organization (WHO) and the Centers for Disease Control and Prevention (CDC), to help guide diagnosis

In [277]:
best_doc=pd.DataFrame(data={'best_docs':best_doc})
best_doc.to_csv('best_documents.csv')

In [291]:
queries='''Range of incubation periods for the disease in humans (and how this varies across age and health status) and how long individuals are contagious, even after recovery.
Prevalence of asymptomatic shedding and transmission. 
Prevalence of asymptomatic shedding and transmission in children, infants, and young people.
Seasonality of transmission of the virus.
Physical science of the coronavirus (e.g., charge distribution, adhesion to hydrophilic or hydrophobic surfaces, environmental survival to inform decontamination efforts for affected areas and provide information about viral shedding).
Persistence and stability on a multitude of substrates and sources (nasal discharge, sputum, urine, fecal matter, blood, bodily fluids and secretions).
Persistence of virus on surfaces of different materials (copper, stainless steel, plastic).
Natural history of the virus and shedding of it from an infected person.
Implementation of diagnostics and products to improve clinical processes.
Disease models, including animal models for infection, disease and transmission.
Tools and studies to monitor phenotypic change and potential adaptation of the virus.
Immune response and immunity to the virus.
Effectiveness of movement control strategies to prevent secondary transmission in health care and community settings.
Effectiveness of personal protective equipment (PPE) and its usefulness to reduce risk of transmission in health care and community settings.
Role of the environment in transmission.'''

In [292]:
query_list = queries.splitlines()
query_list

['Range of incubation periods for the disease in humans (and how this varies across age and health status) and how long individuals are contagious, even after recovery.',
 'Prevalence of asymptomatic shedding and transmission. ',
 'Prevalence of asymptomatic shedding and transmission in children, infants, and young people.',
 'Seasonality of transmission of the virus.',
 'Physical science of the coronavirus (e.g., charge distribution, adhesion to hydrophilic or hydrophobic surfaces, environmental survival to inform decontamination efforts for affected areas and provide information about viral shedding).',
 'Persistence and stability on a multitude of substrates and sources (nasal discharge, sputum, urine, fecal matter, blood, bodily fluids and secretions).',
 'Persistence of virus on surfaces of different materials (copper, stainless steel, plastic).',
 'Natural history of the virus and shedding of it from an infected person.',
 'Implementation of diagnostics and products to improve cl

In [299]:
queries_df = pd.DataFrame(data=[{'Query':query} for query in query_list])
queries_df

,Query
0,Range of incubation periods for the disease in...
1,Prevalence of asymptomatic shedding and transm...
2,Prevalence of asymptomatic shedding and transm...
3,Seasonality of transmission of the virus.
4,"Physical science of the coronavirus (e.g., cha..."
5,Persistence and stability on a multitude of su...
6,Persistence of virus on surfaces of different ...
7,Natural history of the virus and shedding of i...
8,Implementation of diagnostics and products to ...
9,"Disease models, including animal models for in..."


In [294]:
query_vector_list = []
for i in tqdm(range(len(query_list))):
    doc = nlp(query_list[i])
    vec = doc.vector
    query_vector_list.append([i,vec])
query_vector_df = pd.DataFrame(data=query_vector_list,columns=['index','query_vec'])


100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [00:01<00:00,  9.10it/s]


In [300]:
query_vector_df['query_vec'][0].shape
query_list
# so for each query you have a corresponding vectors generated by using en_core_sci_lg model from scispacy

['Range of incubation periods for the disease in humans (and how this varies across age and health status) and how long individuals are contagious, even after recovery.',
 'Prevalence of asymptomatic shedding and transmission. ',
 'Prevalence of asymptomatic shedding and transmission in children, infants, and young people.',
 'Seasonality of transmission of the virus.',
 'Physical science of the coronavirus (e.g., charge distribution, adhesion to hydrophilic or hydrophobic surfaces, environmental survival to inform decontamination efforts for affected areas and provide information about viral shedding).',
 'Persistence and stability on a multitude of substrates and sources (nasal discharge, sputum, urine, fecal matter, blood, bodily fluids and secretions).',
 'Persistence of virus on surfaces of different materials (copper, stainless steel, plastic).',
 'Natural history of the virus and shedding of it from an infected person.',
 'Implementation of diagnostics and products to improve cl

In [304]:
# now you waana find bm25 score of query with each abstract and arrange in rank
Abstract

['Community-ARI and inpatient-ARI was also combined with mild and severe cases of influenza from a historical prospective study as mild-ARI and severe-ARI respectively to evaluate the performance of clinical case definitions. We analysed 130 community-ARI and 140 inpatient-ARI episodes (5 inpatient-ARI excluded because multiple pathogens were detected), involving 138 and 207 samples respectively. Detection by PCR declined with days post-onset for influenza virus; decrease was faster for community-ARI than for inpatient-ARI. No such patterns were observed for noninfluenza respiratory virus infections. PathChip added substantially to viruses detected for community-ARI only. Clinical case definitions discriminated influenza from other mild-ARI but performed poorly for severe-ARI and for older participants. Rational strategies for diagnosis and surveillance of influenza and other respiratory virus must acknowledge the differences between ARIs presenting in community and hospital settings.'

In [324]:
relevance_score = []
best_docs=[]
final_query_list=[]
for i,query in enumerate(query_list):
    final_list=[]
    relevanc_score = bm25.get_scores(query.split(' '))
    tokenize_query = query.split(' ')
    print(tokenize_query)
    relevance_score.append(relevanc_score)
    best_doc = bm25.get_top_n(tokenize_query,Abstract,n=10)
    final_list.append([(relevance,best) for relevance,best in zip(relevanc_score,best_doc)])
    final_query_list.append('QUERY:{} -{}-  Relevance and Best Docs  -{}'.format(i,query,final_list))


['Range', 'of', 'incubation', 'periods', 'for', 'the', 'disease', 'in', 'humans', '(and', 'how', 'this', 'varies', 'across', 'age', 'and', 'health', 'status)', 'and', 'how', 'long', 'individuals', 'are', 'contagious,', 'even', 'after', 'recovery.']
['Prevalence', 'of', 'asymptomatic', 'shedding', 'and', 'transmission.', '']
['Prevalence', 'of', 'asymptomatic', 'shedding', 'and', 'transmission', 'in', 'children,', 'infants,', 'and', 'young', 'people.']
['Seasonality', 'of', 'transmission', 'of', 'the', 'virus.']
['Physical', 'science', 'of', 'the', 'coronavirus', '(e.g.,', 'charge', 'distribution,', 'adhesion', 'to', 'hydrophilic', 'or', 'hydrophobic', 'surfaces,', 'environmental', 'survival', 'to', 'inform', 'decontamination', 'efforts', 'for', 'affected', 'areas', 'and', 'provide', 'information', 'about', 'viral', 'shedding).']
['Persistence', 'and', 'stability', 'on', 'a', 'multitude', 'of', 'substrates', 'and', 'sources', '(nasal', 'discharge,', 'sputum,', 'urine,', 'fecal', 'matter

In [328]:
relevance_score

[array([25.82029577, 24.20145047, 26.46841634, ..., 34.55005646,
        27.42105953, 26.64617615]),
 array([9.25661968, 7.14180108, 9.79911544, ..., 9.73177224, 9.71088757,
        9.99259363]),
 array([17.09976917, 19.37097277, 19.50553372, ..., 18.70731354,
        18.81471524, 18.7648086 ]),
 array([12.13057163, 14.23949642, 14.00445766, ..., 15.49246429,
        14.69190783, 14.34517783]),
 array([25.43401788, 24.04352027, 22.94707841, ..., 26.16100153,
        23.95851564, 28.35052183]),
 array([22.15532094, 19.26457919, 26.12688212, ..., 24.18930985,
        26.22363782, 24.52118884]),
 array([ 9.2737271 , 11.79311365, 12.79717489, ..., 13.54243984,
        12.99416484, 11.69352537]),
 array([21.78615046, 20.69352054, 22.22662643, ..., 23.33833046,
        21.65947281, 21.60947544]),
 array([14.96732013, 11.10533669,  9.79911544, ..., 13.32562505,
        13.07260089, 13.02891508]),
 array([10.04764257,  4.98097703,  7.42735979, ..., 10.03308669,
         6.57449658,  5.18531983

In [308]:
final_query_list

['QUERY:0 -Range of incubation periods for the disease in humans (and how this varies across age and health status) and how long individuals are contagious, even after recovery.-  Relevance and Best Docs  -[[(25.820295773190928, \'Respiratory viruses cause disease in humans characterized by an abrupt onset of symptoms. Studies in humans and animal models have shown that symptoms are not immediate and appear days or even weeks after infection. Since the initial symptoms are a manifestation of virus recognition by elements of the innate immune response, early virus replication must go largely undetected. The interval between infection and the emergence of symptoms is called the incubation period and is widely used as a clinical score. While incubation periods have been described for many virus infections the underlying mechanism for this asymptomatic phase has not been comprehensively documented. Here we review studies of the interaction between human pathogenic respiratory RNA viruses a

In [309]:
# now combine cosine similarity and bm25 scores for each query, where cosine similarity is the similarity of query vectors generated
# from en_core_sci_lg model and list of documents.
# So basically cosine similarity is the similarity of word vectors of query and documents
from scipy.spatial import distance
# we have query vectors(embeddings) generated from en_core_sci_lg model
query_vector_df

,index,query_vec
0,0,"[-0.034897946, -0.036800146, 0.07446235, -0.01..."
1,1,"[0.01392266, -0.03347473, -0.0377267, -0.16889..."
2,2,"[-0.037775896, 0.008645175, 0.018933149, -0.06..."
3,3,"[0.02576652, -0.12867895, -0.01963384, -0.1463..."
4,4,"[0.043046653, 0.0024149886, 0.013799003, -0.01..."
5,5,"[0.013294059, 0.041299053, 0.075669244, -0.003..."
6,6,"[0.1755125, 0.00026250936, 0.09548046, -0.0730..."
7,7,"[0.061420526, -0.029005961, 0.03332932, -0.137..."
8,8,"[-0.100062, -0.02964807, 0.11236312, -0.015666..."
9,9,"[0.021280766, 0.01191039, 0.052053664, -0.1018..."


In [313]:
doc_vectors = []
for i,text in enumerate(Abstract):
    doc = nlp(text)
    doc_vector = doc.vector
    doc_vectors.append(doc_vector)
    
df_doc_vectors = pd.DataFrame(data={'vectors':doc_vectors})

In [314]:
df_doc_vectors.to_csv('df_doc_vectors.csv')

In [335]:
np.ones((1,19815))[0]

array([1., 1., 1., ..., 1., 1., 1.])

In [355]:
distances = distance.cdist([value for value in query_vector_df['query_vec']],[doc_value for doc_value in df_doc_vectors['vectors']],metric='cosine')
# now take ratio of 0.9 as a weight to both scores (bm25 and cosine similarity)
final_best_docs_list=[]
for i in range(len(query_list)):
    print('\nQUERY:{} '.format(i+1),query_list[i])
    df_doc_vectors['bm25_cosine'] = 0.9*(np.ones((1,19815))[0]-distances[i])+ 0.1*np.array(relevance_score)[i]
    # now find top 10 documents for given queries in 19815 documents
    top_10_indices=np.argsort(df_doc_vectors['bm25_cosine'])[::-1][:10]
    # corresponding docs
    best_docs_diff = [Abstract[i] for i in top_10_indices]
    for i,index in enumerate(top_10_indices):
        print('\nRANK:{}'.format(i+1),'\nSCORE:{}\n'.format(df_doc_vectors['bm25_cosine'][index]),best_docs_diff[i])
    print('-'*101)
    


QUERY:1  Range of incubation periods for the disease in humans (and how this varies across age and health status) and how long individuals are contagious, even after recovery.

RANK:1 
SCORE:6.247581177926511
 Respiratory viruses cause disease in humans characterized by an abrupt onset of symptoms. Studies in humans and animal models have shown that symptoms are not immediate and appear days or even weeks after infection. Since the initial symptoms are a manifestation of virus recognition by elements of the innate immune response, early virus replication must go largely undetected. The interval between infection and the emergence of symptoms is called the incubation period and is widely used as a clinical score. While incubation periods have been described for many virus infections the underlying mechanism for this asymptomatic phase has not been comprehensively documented. Here we review studies of the interaction between human pathogenic respiratory RNA viruses and the host with a p


RANK:1 
SCORE:6.890253443462967
 Purpose: The recent Ebola epidemic that devastated West Africa has infected and killed more health care providers than any other outbreak in the history of this virus. An improved understanding of pathogen transmission and the institution of strategies to protect health care providers against infection are needed in infectious disease outbreaks. This review connects what is known about Ebola virus transmission with personal protective equipment (PPE) designed to arrest nosocomial transmission. Methods: Articles pertaining to filovirus transmission and PPE in filovirus outbreaks were reviewed and findings are presented. In addition, studies that evaluated PPE and donning and doffing strategies are presented. Findings: PPE is one step in a comprehensive infection prevention and control strategy that is required to protect health care providers. Given that the Ebola virus is primarily transmitted through direct contact of mucous membranes and cuts in the 

In [340]:
# APPROACH 
# BM25 -> input->context , documents -> ouput-> top 10 documents
# then feed these top 10 documents one by one as a context in input in bert model with question
df_doc_vectors

,vectors,bm25_cosine
0,"[-0.018040596, -0.0009771533, 0.0878012, -0.03...",1.637599
1,"[0.022314087, -0.010782369, 0.09549482, -0.038...",2.033188
2,"[0.07119918, 0.014441058, 0.09192496, -0.03011...",1.903528
3,"[-0.014332192, 0.013354345, 0.11959983, -0.019...",1.808468
4,"[-0.030337457, -0.0039113234, 0.10172218, -0.0...",1.970253
...,...,...
19810,"[0.034630407, -0.0033983863, 0.100898914, -0.0...",2.044594
19811,"[-0.024249347, -0.014972796, 0.11091785, -0.06...",1.898349
19812,"[0.020668857, 0.0089697065, 0.08878848, -0.019...",2.110961
19813,"[0.0075993054, 0.010085838, 0.13038032, 0.0081...",1.985102


In [343]:
df_doc_vectors.to_csv('df_doc_vectors.csv') # for last query in query_list

In [ ]:
# save these docs
pd.DataFrame(data={'best_docs_diff':best_docs_diff}).to_csv('best_docs_diff.csv') # saving in csv format